In [1]:
import sys
sys.path.append('..')

In [2]:
import modules.initialize as minit

minit.initialize(verbose=True)

Detected CUDA files, patching ldflags
Emitting ninja build file /storage/hdd1/jheuristic/exp/decentralized/jheuristic/adapters/mytorchcudamodules/notebooks/../csrc/build/build.ninja...
Building extension module fused_mix_prec_layer_norm_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module fused_mix_prec_layer_norm_cuda...


In [3]:
from modules import MixedFusedLayerNorm
import torch

In [4]:
device = torch.device('cuda')

In [5]:
sz = [2, 4096, 4096]
layer = MixedFusedLayerNorm(sz[-1]).to(device)
input = torch.zeros(sz).to(device).unsqueeze(0)
input = input.unsqueeze(0)

out = layer(input)
print('forward', out)
print('backward', out.backward(torch.randn_like(out)))

forward tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]], device='cuda:0',
       grad_fn=<FusedLayerNormAffineFunctionBackward>)
backward None


In [6]:
input = torch.randn(sz).to(device).unsqueeze(0)


In [14]:
%%time
for i in range(1000):
    out = layer(input)
    out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 1.71 s, sys: 912 ms, total: 2.62 s
Wall time: 2.65 s


In [17]:
%%time
with torch.cuda.amp.autocast():
    for i in range(1000):
        out = layer(input)
        out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 1.65 s, sys: 958 ms, total: 2.61 s
Wall time: 2.66 s


### Baseline

In [1]:
import torch, torch.nn as nn
sz = [2, 4096, 4096]
device='cuda:0'
layer = nn.LayerNorm(sz[-1]).to(device)
input = torch.zeros(sz).to(device).unsqueeze(0)
input = input.unsqueeze(0)

out = layer(input)
print('forward', out)
print('backward', out.backward(torch.randn_like(out)))

forward tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]]]], device='cuda:0',
       grad_fn=<NativeLayerNormBackward>)
backward None


In [2]:
input = torch.randn(sz).to(device).unsqueeze(0)


In [4]:
%%time
for i in range(1000):
    out = layer(input)
    out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 943 ms, sys: 549 ms, total: 1.49 s
Wall time: 1.52 s


In [6]:
%%time
with torch.cuda.amp.autocast():
    for i in range(1000):
        out = layer(input)
        out.backward(torch.randn_like(out))
torch.cuda.synchronize()

CPU times: user 900 ms, sys: 611 ms, total: 1.51 s
Wall time: 1.52 s
